In [96]:
## imports
import importlib
import osmosys.osmosys
import activityinfo
import pandas as pd
import model.modelAI
from IPython.display import display
import json
import os.path
from urllib.error import HTTPError

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo)

<module 'activityinfo' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\activityinfo\\__init__.py'>

In [81]:
month = 'JUNIO'

In [ ]:
osmosysData = osmosys.osmosys.get_ai_form_level_1()
print(osmosysData.info())

In [19]:
# get all unique form ids
#osmosysData.form_id.value_counts()
parentFormsIds = osmosysData.parent_form_id.unique()
print(parentFormsIds)
print(type(parentFormsIds))
print(osmosysData.groupby(['parent_form', 'parent_form_id']).size())

['cx2rbbnl4q3ommxda' 'chmqarpl4q3ommxcz' 'cr62ru9l4q3ommxdt'
 'cq0tf6fl4q3ommxdu' 'cqc8ug2l4q3ommxdz' 'cf6k80l4q3ommxdp'
 'culktyl4q3ommxdr' 'c41ibpyl4q3ommxdo']
<class 'numpy.ndarray'>
parent_form               parent_form_id   
Agua saneamiento higiene  culktyl4q3ommxdr      5
Alojamiento               c41ibpyl4q3ommxdo    26
Educación                 cx2rbbnl4q3ommxda    33
Integración               chmqarpl4q3ommxcz    47
Protección VBG            cqc8ug2l4q3ommxdz    50
Protección a la infancia  cr62ru9l4q3ommxdt    37
Protección general        cq0tf6fl4q3ommxdu    61
Salud                     cf6k80l4q3ommxdp      3
dtype: int64


In [49]:
importlib.reload(osmosys.osmosys)
aiToken = osmosys.osmosys.getToken()
formTotalDf = None
for parentFormsId in parentFormsIds:
    print(parentFormsId)
    client = activityinfo.Client(token=aiToken, base_url='https://www.activityinfo.org/resources')
    dbsJson = client.get_resource('form/{parentFormsId}/query'.format(parentFormsId=parentFormsId))
    dbsDf = pd.DataFrame.from_records(dbsJson)
    dbsDf['parent_form_id'] = parentFormsId
    print(dbsDf.shape)
    if formTotalDf is None:
        formTotalDf = dbsDf
    else:
        formTotalDf = pd.concat([formTotalDf, dbsDf], axis=0)
    print(formTotalDf.shape)

cx2rbbnl4q3ommxda
(89, 33)
(89, 33)
chmqarpl4q3ommxcz
(252, 33)
(341, 34)
cr62ru9l4q3ommxdt
(204, 33)
(545, 35)
cq0tf6fl4q3ommxdu
(332, 33)
(877, 36)
cqc8ug2l4q3ommxdz
(216, 33)
(1093, 37)
cf6k80l4q3ommxdp
(292, 33)
(1385, 38)
culktyl4q3ommxdr
(151, 33)
(1536, 39)
c41ibpyl4q3ommxdo
(335, 33)
(1871, 40)


In [50]:
# ordeno para quitar duplicados
# filtro implementaciòn indirecta y doante acnur
print('formTotalDf: ' + str(formTotalDf.shape))
formPartnersDf = formTotalDf.loc[
    (formTotalDf['implementacion'] == 'Indirecta') & (formTotalDf['donante.@id'] == 'chgxs46l02wy5i81e')]
print('formPartnersDf: ' + str(formPartnersDf.shape))

formTotalDf: (1871, 40)
formPartnersDf: (350, 40)


In [51]:
formPartnersDfClean = formPartnersDf.sort_values('@lastEditTime', ascending=False).drop_duplicates(
    subset=['implementador.@id', 'ubicacion.@id', 'parent_form_id'])
print('formPartnersDfClean: ' + str(formPartnersDfClean.shape))
formPartnersDfClean = formPartnersDfClean[
    ["@id", "@lastEditTime", "implementacion", "donante.@id", "donante.org_nombre", "donante.donante",
     "implementador.@id",
     "implementador.org_nombre", "implementador.donante", "ubicacion.@id", "ubicacion.name", "ubicacion.code",
     "ubicacion.parent.name", "ubicacion.parent.code", "codigo_referencia", "parent_form_id"
     ]]

formPartnersDfClean: (284, 40)


In [74]:
## hago merge para buscar cuales falta por crear
mergeDf = pd.merge(osmosysData, formPartnersDfClean, how='left',
                   left_on=['parent_form_id', 'id_registro_imp', 'canton_id_registro'],
                   right_on=['parent_form_id', 'implementador.@id', 'ubicacion.@id'])
formsToCreate = mergeDf[mergeDf['@id'].isna()]
numberOfFormTocreate = formsToCreate.shape[0]
print('Forms to create: ' + str(numberOfFormTocreate))

Forms to create: 0


In [94]:
importlib.reload(model.modelAI)
formLists = []
for index, row in formsToCreate.iterrows():
    form = model.modelAI.Form('40038', 'cl5aagbkyyk18of2', 'c89eykqkykc8oig1b', 'chgxs46l02wy5i81e',
                              row['id_registro_imp'],
                              row['canton_id_registro'])
    formLists.append(form)

## creo las estructuras de datos
changesList = []

for index, row in formsToCreate.iterrows():
    form = model.modelAI.Form('40038', 'cl5aagbkyyk18of2', 'c89eykqkykc8oig1b', 'chgxs46l02wy5i81e',
                              row['id_registro_imp'],
                              row['canton_id_registro'])
    record = model.modelAI.Record(row['parent_form_id'], activityinfo.generate_id(), None, form)
    changesList.append(record)

changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "form_level_1" + month + ".json"), "w")
#write string to file
n = text_file.write(finalJson)
#close file
text_file.close()




{"changes": []}


In [99]:
## send to AI
try:
    if (numberOfFormTocreate > 0):
        client.post_resource('update', body=finalJson)
        print("Enviado a AI")
    else:
        print("nada por crear en AI")
except HTTPError as e:
    code = e.response.status_code
    print('error:')
    print(code)
    print(e)

nada por crear en AI
